# 일급 함수
- 런타임에 생성
- 변수나 자료구조의 원소에 할당됨(객체로 취급됨)
- 함수의 인자로 넘겨질 수 있음
- 함수의 리턴 값으로 사용할 수 있음

### 함수는 오브젝트!

In [1]:
def factorial(n):
    '''returns n!'''
    return 1 if n <2 else n* factorial(n-1)
print(factorial(42))
print(factorial.__doc__)
print(type(factorial))

1405006117752879898543142606244511569936384000000000
returns n!
<class 'function'>


마지막 줄을 보면 `factorial()`은 `function` class의 인스턴스임을 알 수 있다.
# 고차 함수
고차함수는 아래의 두 가지 조건을 만족하는 함수이다.

- 함수의 인자로 사용되거나
- 함수의 리턴값으로 사용됨

ex)`map`, `sorted`, `filter`, `reduce`

`map`, `filter`, `reduce`는 `listcomp`와 `genexp`로 대체된다. 기능은 대체하면서, 더 읽기 편하다. (2-1 참고)

# 익명 함수
`lambda` 키워드는 익명 함수를 생성하는 키워드이다. `lambda`는 할당을 만들거나, `while`, `try` 등의 문에서 사용할 수 없다. 익명 함수를 가장 잘 쓰는 방법은 인자 목록의 context에서 사용하는 것이다.
Example)
``` python
sorted(fruits, key=lambda word: word[::-1])
```

# The seven flavors of callable objects
1. User-defined functions
2. Built-in functions
3. Built-in methods
4. Methods
5. Classes
6. Class instances
7. Generator functions

Python에서 주어진 객체가 callable인지 알아보는 가장 안전한 방법은 `callable()`에 넣어보는 것이다. True나 False를 return한다.

## 사용자 정의 callable types

In [2]:
import random

class BingoCage:
    
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
            
    def __call__(self):
        return self.pick()
    
bingo = BingoCage(range(3))
bingo()

2

마지막 줄을 보면 인스턴스가 callable한 것을 볼 수 있다. `__call__`을 구현하는 클래스는 `BingoCage`에 남아있는 아이템처럼 호출들로부터 보존되야 할 내부 상태를 가지고 있는 함수 같은 객체를 생성하는 쉬운 방법이다. (`__call__`을 쓰라는 말)

## 함수 뜯어보기(?)
`dir`을 사용하면 객체에 들어있는 속성들을 볼 수 있다.

In [5]:
class C: pass
obj = C()
def func(): pass
sorted(set(dir(func)) - set(dir(obj)))

['__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__']

결과를 확인하면 함수 객체와 일반 사용자 정의 객체와 다른 속성들을 확인할 수 있다. 이에 대한 설명은 표 5-1에 간략히 기술되어 있고, 이 중에서 `__defaults__`, `__code__`, `__annotations__`을 더 살펴본다.

## 위치 인자에서 키워드 인자로
이 부분은 다른 예제로 대체합니다.

In [6]:
def f(a, *args, **kwargs):
    print('a=', a)
    print('args=',args)
    print('kwargs=',kwargs)
    
f(11)
f(11,22,33)
f(11,22,33,b=44, c=55)

a= 11
args= ()
kwargs= {}
a= 11
args= (22, 33)
kwargs= {}
a= 11
args= (22, 33)
kwargs= {'b': 44, 'c': 55}


위치 인자부터 순서대로 들어간다. 만약 뒤에 인자가 더 주어진다면 `*args`가 `tuple` 형태로 받아간다. 정의에서 없는 속성값을 넣을 경우 `**kwargs`가 `dict` 형태로 받아간다. 책에 나온 `cls`는 그래서 키워드 인자 강제로만 받을 수 있다고 한 것(keyword-only parameter) \*가 붙은 인자 뒤에 인자를 키워드 인자를 정의하게 되면 키워드 인자 강제가 된다.

# 매개 변수에 대한 정보 검색
`Bobo`가 어떻게 매개 변수에 대한 정보를 찾아낼까?  
함수 객체에는 `__defaults__`가 있다. 여기에는 위치 인자와 키워드 인자의 기본값이 튜플로 저장되어 있다. 키워드 전용 인자는 `__kwdefaults__`에서 보인다. 하지만 인자의 이름은 자체 속성이 많은 code 객체를 참조하고 있는 `__code__`에서 찾을 수 있다.

In [10]:
def clip(text, max_len=80):
    """Return text clipped at the last space before or after max_len
    """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None: # no spaces were found
        end = len(text)
    return text[:end].rstrip()

print(clip.__defaults__)
print(clip.__code__)
print(clip.__code__.co_varnames)
print(clip.__code__.co_argcount)

(80,)
<code object clip at 0x7f024593bc00, file "<ipython-input-10-7c01d158dcd6>", line 1>
('text', 'max_len', 'end', 'space_before', 'space_after')
2


문제점1: \*나 \*\*가 붙은 변수는 포함하지 않음.  
문제점2: default value의 수와 변수의 수가 맞지 않을 경우는?

In [14]:
from inspect import signature
sig = signature(clip)
sig

<Signature (text, max_len=80)>

In [15]:
str(sig)

'(text, max_len=80)'

In [17]:
for name, param in sig.parameters.items():
    print(param.kind, ':', name, '=', param.default)

POSITIONAL_OR_KEYWORD : text = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : max_len = 80


`kind`는 다섯개의 값 중 하나를 가질 수 있다.
 1. POSITIONAL_OR_KEYWORD : 위치 인자이거나, 키워드 인자
 2. VAR_POSITIONAL: 위치 인자 tuple
 3. VAR_KEYWORD: 키워드 인자 dict
 4. KEYWORD_ONLY: 키워드 인자 강제
 5. POSITIONAL_ONLY: 위치 인자 강제
 
`inspect.Signature` 객체는 `bind` 메소드를 가지는데, 간단하게 말하면 `bind` 메소드는 인자들을 받아서 `signature`에 있는 함수들과 묶는다.

# 함수 주석
Python 3는 메타데이터를 매개 변수나, 리턴값에 연결하는 구문을 제공한다.
``` python
def clip(text:str, max_len:'int > 0'=80) -> str:
```
주석의 위치에 주목하자. 기본값이 있을 때의 위치, 없을 때의 위치.  
`-> str`은 리턴값의 type을 의미한다. 이것들은 `__annotations__`에 저장된다.

# 함수형 프로그래밍을 위한 패키지
## operator 모듈

In [22]:
from functools import reduce
from operator import mul

def fact1(n):
    return reduce(lambda a, b: a*b, range(1, n+1))

def fact2(n):
    return reduce(mul, range(1, n+1))

print(fact1(10), fact2(10))

3628800 3628800


### itemgetter, attrgetter

In [23]:
metro_data = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]
from operator import itemgetter
for city in sorted(metro_data, key=itemgetter(1)):
    print(city)
    
cc_name = itemgetter(1, 0)
for city in metro_data:
    print(cc_name(city))

('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))
('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'Sao Paulo')


위의 결과처럼 여러개의 인자를 전달하면 추출된 값을 `tuple`로 리턴한다. `itemgetter`는 [] 연산자를 사용해서 시퀀스 뿐만 아니라 `__getitem__`이 구현된 매핑과 클래스에도 사용할 수 있다.  

`itemgetter`과 비슷한 `attrgetter`는 객체의 속성을 이름으로 추출하여 `tuple`로 반환한다. 인자에 .(dot)을 사용하면 중첩된 객체를 navigating한다. 

In [24]:
from collections import namedtuple
LatLong = namedtuple('LatLong', 'lat long')
Metropolis = namedtuple('Metropolis', 'name cc pop coord')
metro_areas = [Metropolis(name, cc, pop, LatLong(lat, long))
               for name, cc, pop, (lat, long) in metro_data]
metro_areas[0]

Metropolis(name='Tokyo', cc='JP', pop=36.933, coord=LatLong(lat=35.689722, long=139.691667))

In [25]:
metro_areas[0].coord.lat

35.689722

In [26]:
from operator import attrgetter
name_lat = attrgetter('name', 'coord.lat')
for city in sorted(metro_areas, key= attrgetter('coord.lat')):
    print(name_lat(city))

('Sao Paulo', -23.547778)
('Mexico City', 19.433333)
('Delhi NCR', 28.613889)
('Tokyo', 35.689722)
('New York-Newark', 40.808611)


In [29]:
import operator
print([name for name in dir(operator) if not name.startswith('_')])

['abs', 'add', 'and_', 'attrgetter', 'concat', 'contains', 'countOf', 'delitem', 'eq', 'floordiv', 'ge', 'getitem', 'gt', 'iadd', 'iand', 'iconcat', 'ifloordiv', 'ilshift', 'imatmul', 'imod', 'imul', 'index', 'indexOf', 'inv', 'invert', 'ior', 'ipow', 'irshift', 'is_', 'is_not', 'isub', 'itemgetter', 'itruediv', 'ixor', 'le', 'length_hint', 'lshift', 'lt', 'matmul', 'methodcaller', 'mod', 'mul', 'ne', 'neg', 'not_', 'or_', 'pos', 'pow', 'rshift', 'setitem', 'sub', 'truediv', 'truth', 'xor']


앞에 i가 붙은 것들은 in-place 연산을 의미하고 immutable 할 경우, fallback으로 i가 붙지 않은 함수로 작동한다. (Chap.2-1 참고)
### methodcaller
`methodcaller`는 메소드의 이름으로 메소드를 호출한다.

In [30]:
from operator import methodcaller
s = 'The time has come'
upcase = methodcaller('upper')
upcase(s)

'THE TIME HAS COME'

In [31]:
hiphenate = methodcaller('replace', ' ', '-')
hiphenate(s)

'The-time-has-come'

두번째 예시를 보면 `methodcaller`가 인자를 가지고 있는 것을 볼 수 있다. 
### functools.partial으로 인자 고정하기

In [36]:
from operator import mul
from functools import partial
triple = partial(mul, 3)
print(triple(7))
print(list(map(triple, range(1, 10))))

21
[3, 6, 9, 12, 15, 18, 21, 24, 27]


In [34]:
import unicodedata, functools
nfc = functools.partial(unicodedata.normalize, 'NFC')
s1 = 'café'
s2 = 'cafe\u0301'
print(s1, s2)
print(s1 == s2)
print(nfc(s1) == nfc(s2))

café café
False
True
